## Práctica 1. Resolución de problemas con búsqueda 
### Inteligencia Artificial I    2024/2025

Parte 1: AIMA 
Este notebook está organizado en 3 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas de juguete como el de las jarras, el problema de los misioneros y el problema del ocho puzzle. 
* Parte II: se muestra cómo utilizar los algoritmos de búsqueda exhaustiva (ciega) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
Debes entender el comportamiento de la búsqueda en los distintos problemas. 
Observa la representación de los problemas de ejemplo y el comportamiento de los distintos algoritmos. 
**No hay que entregar este notebook** 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en representar estados y acciones, la comprobación de objetivo y el coste de las acciones.
La siguiente clase Problem (de la librería AIMA) representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problem, y requerirá implementar
* estado_inicial, es_estado_final(_), acciones(_), aplica(_,_), coste_de_aplicar_accion y eventualmente __init__ 



!git clone https://github.com/aimacode/aima-python.git
#_Para esta práctica se usarán los archivos _search.py y utils.py_ de AIMA que se pueden descargar del campus_ 

In [3]:
#3cd aima-python

[WinError 2] El sistema no puede encontrar el archivo especificado: 'aima-python'
C:\Users\usuario_local\iA


In [2]:
import sys
sys.path.append('.')
from search import *

ModuleNotFoundError: No module named 'numpy'

In [5]:
# El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 
from search import Problem

## Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver algunos ejemplos de cómo __definir un problema como subclase
de Problem__. En concreto, el problema de las jarras, el problema de los misioneros, y el puzle de 8 vistos en clase.
Observa en cada problema como definir los estados, las acciones, estado inicial y objetivo.

### Problema de las jarras

In [1]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs=list() 
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self,estado,accion):
    # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


NameError: name 'Problem' is not defined

Vamos a probar algunos ejemplos para entender cómo funciona la clase Problem.

In [7]:
# Estado inicial
p =Jarras()
p.initial

(0, 0)

In [8]:
# Acciones válidas desde el estado inicial (actions comprueba las acciones para las que se cumplen las precondiciones). 
# Generador: sólo acciones válidas e interesantes.
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [9]:
# Resultado de aplicar una acción a un estado
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [10]:
# Comprobamos si un estado dado es objetivo
p.goal_test(p.initial)

False

### Problema de los misioneros

In [14]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Aunque no es necesario puedes hacer tu propia implementación del problema 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [15]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

In [16]:
misioneros.initial

(3, 3, 0)

In [17]:
# Acciones aplicables en el estado inicial
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

Por ejemplo, para resolver el problema de los misioneros con 
el método de busqueda en anchura la llamada sería:  


In [18]:
breadth_first_tree_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

In [55]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        pos_hueco = estado.index(0)  # busco la posicion del 0
        accs = list()
        if pos_hueco not in (0, 1, 2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0, 3, 6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (6, 7, 8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (2, 5, 8):
            accs.append("Mover hueco derecha")

        return accs

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        def swap(x):
            l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]
        if accion == "Mover hueco arriba":
            swap(-3)
        elif accion == "Mover hueco abajo":
            swap(3)
        elif accion == "Mover hueco izquierda":
            swap(-1)
        elif accion == "Mover hueco derecha":
            swap(1)

        return tuple(l)
    
    def coste_de_aplicar_accion(self, s, a):
        return 1 

    def check_solvability(self, state):
            """ Checks if the given state is solvable """
    # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
    #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
            inversion = 0
            for i in range(len(state)):
                for j in range(i+1, len(state)):
                    if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                        inversion += 1
            return inversion % 2 == 0   

####  Prueba los siguientes ejemplos.

In [20]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [21]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [27]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [33]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

¿Por qué da error?
Da error porque el hueco se encuentra abajo en el medio, por lo tanto no puede ir hacia abajo ya que está fuera de rango. 

"Cell In[19], line 33, in Ocho_Puzzle.result.<locals>.swap(x)
     32 def swap(x):
---> 33     l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]

IndexError: list index out of ra
nge"
¿Por qué no es necesario añadir controles adicionales para que no ocurra este error cuando estemos usando los algoritmos de búsqued
Porque la funcion actions de la clase Problem de Puzle8 para ver que acciones se pueden ejecutar y cuales no.
a? 


In [31]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [32]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

## Parte II: Experimentación. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad con las variantes vistas en clase) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [35]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search, uniform_cost_search, depth_limited_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [36]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [37]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### A continuación prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8, observando cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [38]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))
# Utiliza los distintos algoritmos de búsqueda ciega para resolver los problemas que hemos representado. 

In [ ]:
from search import *
breadth_first_tree_search(misioneros).solution()

In [43]:
## Problema del puzzle de 8

In [44]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [45]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [46]:
# Definimos esta función auxiliar para facilitar la visualización de los estados
def show(s):
    print(" ".join(map(str,s[:3])))
    print(" ".join(map(str,s[3:6])))
    print(" ".join(map(str,s[6:])))

In [47]:
show(p8.goal)

1 2 3
4 5 6
7 8 0


In [48]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
breadth_first_tree_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [49]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
moves = breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']
s = (2, 4, 3, 1, 5, 6, 7, 8, 0)
show((2, 4, 3, 1, 5, 6, 7, 8, 0))
for action in moves:
    s = estado.result(s,action)
    print()
    show(s)

2 4 3
1 5 6
7 8 0

2 4 3
1 5 0
7 8 6

2 4 3
1 0 5
7 8 6

2 0 3
1 4 5
7 8 6

0 2 3
1 4 5
7 8 6

1 2 3
0 4 5
7 8 6

1 2 3
4 0 5
7 8 6

1 2 3
4 5 0
7 8 6

1 2 3
4 5 6
7 8 0


In [50]:
## Esta llamada no funciona, se queda en [*] 
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  

KeyboardInterrupt: 

Si el método de busqueda en anchura es completo.. ¿la ejecución anterior no debería terminar? ¿qué crees que está pasando?

No, que el algoritmo de búsqueda en profundidad (dfs) al no estar controlando los repetidos, en el árbol de búsqueda hay ciclos y se queda en un bucle sin fin.


Añade a la definición del problema Ocho_Puzzle la función siguiente check_solvability que comprueba si un estado inicial tiene solución.

En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución.

In [53]:
def check_solvability(self, state):
        """ Checks if the given state is solvable """
# The solvability of a configuration can be checked by calculating the Inversion Permutation. 
#If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0        

In [56]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [86]:
sol

'Este estado inicial no tiene solucion'

In [57]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [58]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [59]:
breadth_first_tree_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [64]:
len(breadth_first_tree_search(estado).solution())
 #Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

8

In [62]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [67]:
# Comprueba que para el mismo estado en este problema el coste del uso de búsqueda en profundidad es inabordable. ¿Puedes explicar la razón?
# En la parte II veremos cuántos nodos se expanden en esta búsqueda, qué longitud tiene la solución devuelta por este algoritmo y cuánto tiempo tarda en la búsqueda-. 
# Esta llamada SI termina pero puedes detener la ejecución si tarda mucho. 
depth_first_graph_search(estado).solution()


KeyboardInterrupt: 

In [ ]:
# Esta llamada no termina. Asegurate de que entiendes por qué. 
# depth_first_tree_search(estado).solution()

In [71]:
depth_limited_search(estado,8).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

Observa la salida del algoritmo de profundidad limitada y la del algoritmo de búsqueda en anchura. ¿Son iguales? ¿Es la solución óptima? ¿Es el resultado esperado?

No son iguales pero dan la misma solución con el resultado esperado.

In [69]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

Observa los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega. 
¿Qué algoritmo es más eficiente? 
El algortimo de anchura, breadth_first_graph_search.
¿encuentran la solución óptima? 
El de profundidad limitada no encuentra la solución óptima, pero el de anchura si.

¿son completos? 
Si, los dos

¿alguno no ha podido terminar la ejecución? ¿Por qué? 
Los dos han terminado la ejecución, porque han sido capaces de encontrar la solución.



## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



In [72]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [77]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8 = Ocho_Puzzle(estado_inicial)
p8p=Problema_con_Analizados(p8)

In [78]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [75]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [76]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
breadth_first_tree_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [79]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [80]:
e1 = (7,2,4,3,6,1,8,0,5)

In [88]:
%%time
resuelve_ocho_puzzle(e1,depth_first_graph_search)

Este problema no tiene solucion. 
CPU times: total: 0 ns
Wall time: 999 µs


In [89]:
e2=(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [90]:
resuelve_ocho_puzzle(e2,breadth_first_tree_search)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 8. Nodos analizados: 2017


In [91]:
# Piensa por qué tarda tanto la ejecución siguiente. ¿Cuántos nodos ha analizado?
Ejecuta 130988 nodos con longitud de solución 53510.
# Esta es una solución: Longitud de la solución: 53510. Nodos analizados: 130988 
# ¿se obtiene siempre la misma? ¿es óptima?
No es óptima, porque usa búsqueda en profundidad.

In [ ]:
%%time
resuelve_ocho_puzzle(e2,depth_first_graph_search)

In [93]:
%%time
resuelve_ocho_puzzle(e2,breadth_first_tree_search)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 8. Nodos analizados: 2017
CPU times: total: 0 ns
Wall time: 9.98 ms
